# Analyze particles

This notebook demonstrates how to analyze a batch of images associated to the paper ['Subdiffraction imaging of centrosomes reveals higher-order organizational features of pericentriolar material.'](http://dx.doi.org/10.1038/ncb2591) using the scripting facility available in [Fiji](https://fiji.sc/)

Fiji has been installed with few other plugins including the omero_ij plugin to allow to connect to an OMERO server.

## Description

The following section shows:
 * how to connect to OMERO
 * how to open an OMERO image using Bio-Formats
 * how to analyse an image using Fiji

## Setup: start the [desktop](../../desktop) if it is not already up
The link should open in a different window. If you see an error message try refreshing the window.

## Setup: Add plugins to Classpath
First we had Fiji and the plugins to the classpath.

In [1]:
//Add dependencies to the classpath
%classpath add jar /srv/conda/vnc/Fiji.app/jars/*
%classpath add jar /srv/conda/vnc/Fiji.app/plugins/*

### Connection information 

In [2]:
//Connection information
HOST = "wss://idr.openmicroscopy.org/omero-ws"
USERNAME = "public"
PASSWORD = "public"

public

### Connect to OMERO

In [3]:
import omero.gateway.Gateway
import omero.gateway.LoginCredentials
import omero.log.SimpleLogger

// Method to connect to OMERO
def connect_to_omero(host, user, password) {
    "Connect to OMERO"

    credentials = new LoginCredentials()
    credentials.getServer().setHostname(host)
    credentials.getUser().setUsername(user.trim())
    credentials.getUser().setPassword(password.trim())
    simpleLogger = new SimpleLogger()
    gateway = new Gateway(simpleLogger)
    gateway.connect(credentials)
    return gateway

}

// Connect to OMERO
println "connecting..."
gateway = connect_to_omero(HOST, USERNAME, PASSWORD)
println "connected..."


connecting...
connected...


null

## Select an image . Do not re-run this cell.

Select an image from the [idr0021 study](https://idr.openmicroscopy.org/search/?query=Name:idr0051).

In [4]:
//Move to the next cell after entering a value
ga = new EasyForm("Select the Image to analyze")
ga.addTextField("ImageID").onInit({ga['ImageID'] = "1884807"})
ga

## Collect parameters 

In [5]:
//Image to analyze
image_id = ga['ImageID'].toLong()
group_id = "-1"

-1

## Open the image using Bio-Formats

In [7]:
import ij.IJ
import ij.plugin.frame.RoiManager

//Function to Open an OMERO image using Bio-Formats
def open_image_plus(host, username, password, group_id, image_id) {
    "Open the image using the Bio-Formats Importer"

    StringBuilder options = new StringBuilder()
    options.append("location=[OMERO] open=[omero:server=")
    options.append(host)
    options.append("\nuser=")
    options.append(username)
    options.append("\nport=")
    options.append(443)
    options.append("\npass=")
    options.append(password)
    options.append("\ngroupID=")
    options.append(group_id)
    options.append("\niid=")
    options.append(image_id)
    options.append("] ")
    options.append("windowless=true view=Hyperstack ")
    IJ.runPlugIn("loci.plugins.LociImporter", options.toString())
}

println "opening Image..."
// Open the Image using Bio-Formats
open_image_plus(HOST, USERNAME, PASSWORD, group_id, String.valueOf(image_id))

opening Image...


loci.plugins.LociImporter@5960cad2

## Analyze the image

We run [fiji-macro-segment.ijm](../scripts/fiji-macro-segment.ijm) on the image.

The macro file will:
 - convert the image into an 8-bit image
 - set the autothreshold to MaxEntropy
 - run the Analyze Particles plugin

In [8]:
import ij.IJ
import ij.plugin.frame.RoiManager

imp = IJ.getImage()
RoiManager.getRoiManager()
IJ.runMacroFile("../scripts/fiji-macro-segment.ijm")
println "Analysis completed"

 	Area	Mean	Min	Max
1	0.154	144.031	22	188
2	0.016	175.300	162	186
3	0.050	153.188	95	189
4	0.121	126.701	0	189
5	0.036	164.870	128	189
6	0.240	121.431	0	189
7	0.176	142.321	15	189
8	0.017	180.364	171	189
9	0.031	137.600	57	189
10	0.019	174.250	153	189
11	0.019	167.917	144	187
12	0.025	168.500	148	188
13	0.017	167.091	142	187
14	0.315	143.085	0	189
15	0.198	144.730	49	189
16	0.049	169.645	147	187
17	0.022	164.500	140	183
Analysis completed


null

## Automate

We now run the macro on all the images in the study. This will take several minutes.
This is mainly to show how to iterate over all the images in a study, you can also have a look at [idr0021.groovy](../scripts/idr0021.groovy).

In [9]:
project_id = 51

51

### Analyze the images

In [ ]:
import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility
import ij.IJ

//helper functions
//Function to Open an OMERO image using Bio-Formats
def open_image_plus(host, username, password, group_id, image_id) {
    "Open the image using the Bio-Formats Importer"

    StringBuilder options = new StringBuilder()
    options.append("location=[OMERO] open=[omero:server=")
    options.append(host)
    options.append("\nuser=")
    options.append(username)
    options.append("\nport=")
    options.append(443)
    options.append("\npass=")
    options.append(password)
    options.append("\ngroupID=")
    options.append(group_id)
    options.append("\niid=")
    options.append(image_id)
    options.append("] ")
    options.append("windowless=true view=Hyperstack ")
    IJ.runPlugIn("loci.plugins.LociImporter", options.toString())
}

//Load the datasets
def get_datasets(gateway, ctx, project_id) {
    "List all datasets contained in a Project"

    browse = gateway.getFacility(BrowseFacility)
    ids = new ArrayList(1)
    ids.add(new Long(project_id))
    projects = browse.getProjects(ctx, ids)
    return projects[0].getDatasets()
}

//Load the images
def get_images(gateway, ctx, dataset_id) {
    "List all images contained in a Dataset"

    browse = gateway.getFacility(BrowseFacility)
    ids = new ArrayList(1)
    ids.add(new Long(dataset_id))
    return browse.getImagesForDatasets(ctx, ids)
}

user = gateway.getLoggedInUser()
group_id = user.getGroupId()
ctx = new SecurityContext(group_id)

datasets = get_datasets(gateway, ctx, project_id)

//Close windows before starting
IJ.run("Close All")
datasets.each() { d ->
    dataset_name = d.getName()
    // for each dataset load the images
    // get all images_ids in the dataset
    images = get_images(gateway, ctx, d.getId())
    images.each() { image ->
        if (image.getName().endsWith(".tif")) {
            return
        }
        id = image.getId()
        println "Running macro on image: "+id
        open_image_plus(HOST, USERNAME, PASSWORD, group_id, id)
        imp = IJ.getImage()
        IJ.runMacroFile("../scripts/fiji-macro-segment.ijm")
    }
}

Running macro on image: 1884856
 	Area	Mean	Min	Max
1	0.027	128.588	43	184
2	0.020	125.692	51	181
3	0.016	148.300	110	178
4	0.017	143.636	96	184
5	0.022	130.357	69	178
6	0.030	130.368	41	181
7	0.017	157.091	125	181
8	0.449	115.531	0	184
9	0.016	134.000	83	181
10	0.016	136.400	93	172
11	0.016	146.800	110	184
12	0.025	130.188	51	183
13	0.019	134.417	72	183
14	0.019	126.917	70	171
15	0.022	149.143	103	178
16	0.020	138.385	76	183
17	0.030	104.789	0	181
18	0.016	147.300	111	178
19	0.027	149.059	100	183
20	0.025	161.312	128	184
21	0.017	160.455	134	181
22	0.047	138.733	77	183
23	0.023	128.333	62	173
24	0.016	160.500	130	184
Running macro on image: 1884857
 	Area	Mean	Min	Max
1	0.208	154.714	0	201
2	0.237	149.822	31	201
3	0.020	158.769	113	187
4	0.020	166.769	130	201
5	0.020	169.077	131	194
6	0.023	183.533	156	200
7	0.066	160.000	127	201
8	0.044	120.286	0	200
9	0.031	184.850	154	201
10	0.036	163.522	97	198
11	0.051	166.333	120	200
12	0.022	156.643	105	199
Running macro on image: 1884860
 	Are

## Close the connection

In [10]:
// Close the connection
gateway.disconnect()
println "Done"

Done


true